In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import gc
import cv2
import random
import argparse
import numpy as np
import pandas as pd
from glob import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm.autonotebook import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import wandb

from utils_.set_path import *
from utils_.set_seed import seed_everything
from utils_.loss import FocalLoss, LabelSmoothingLoss, F1Loss
from utils_.get_class_weight import calc_class_weight
from utils_.sam import SAM
from runner.pytorch_timm import TimmModel
from runner.train_runner import CustomTrainer
from data.dataset import CustomTrainDataset

In [3]:
def main(args, train_df, data_dir, model_name, num_classes):
    gc.collect() # python 자원 관리 
    torch.cuda.empty_cache() # gpu 자원관리

    if model_name == 'Gender':
        drop_label = ['006424', '006339', '003713', '003437', '003421', '003399', '003294', '003169', '003113', '003014', '001520', '001266', '000725', '000647']
        drop_label_index = [train_df[train_df['id']==dl].index[0] for dl in drop_label]
        for index in drop_label_index:
            train_df = train_df.drop(index)            
        train_, val_ = train_test_split(train_df, test_size=0.2, random_state=args.seed, stratify=train_df['gender_label'])
    elif model_name == 'Age':
        train_, val_ = train_test_split(train_df, test_size=0.2, random_state=args.seed, stratify=train_df['age_label'])
    else:
        train_, val_ = train_test_split(train_df, test_size=0.2, random_state=args.seed)

    print('='*25, f'{model_name} Model Train Start', '='*25)
    
    # -- dataset & dataloader
    train_dataset = CustomTrainDataset(model_name, data_dir, train_['path'].values, args.resize, transforms=True)
    train_dataloader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=True, num_workers=8)
    
    val_dataset = CustomTrainDataset(model_name, data_dir, val_['path'].values, args.resize, transforms=False)
    valid_dataloader = DataLoader(val_dataset, batch_size = args.batch_size, shuffle=False, num_workers=8)

    model = TimmModel(args, num_classes=num_classes, pretrained=True).to(device)
    model = nn.DataParallel(model)
    
    class_weight = calc_class_weight(train_dataset, model_name, num_classes, device)
    # criterion = FocalLoss(weight=class_weight).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weight).to(device)
    
    base_optim = torch.optim.AdamW 
    # optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-3)
    optimizer = SAM(model.parameters(), base_optim, lr=args.lr, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-9, verbose=True)

    trainer = CustomTrainer(args, model=model, train_dataloader=train_dataloader, valid_dataloader=valid_dataloader, optimizer=optimizer, scheduler=scheduler, criterion=criterion, device=device)
    model, best_score, best_loss = trainer.train(model_name)
    
    print('='*25, f'{model_name} Model Train End', '='*25)
    
    return model, best_score, best_loss

## Config

In [4]:
parser = argparse.ArgumentParser()
    
# Data and model checkpoints directories
parser.add_argument('--seed', type=int, default=909)
parser.add_argument('--epochs', type=int, default=100)
parser.add_argument("--resize", nargs="+", type=int, default=[512, 384])
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--model', type=str, default='resnet34')
parser.add_argument('--lr', type=float, default=1e-6)
parser.add_argument('--criterion', type=str, default='CE')
parser.add_argument('--patience', type=int, default=5)

args = parser.parse_args(args=[])

In [5]:
seed_everything(args.seed)    
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Set Directory & Log

In [6]:
data_dir = TRAIN_IMG_FOLDER_PATH
model_dir = MODEL_SAVE_PATH    
project_idx = len(glob('/workspace/models_all/*')) + 1
os.makedirs(f'{model_dir}_all/Project{project_idx}', exist_ok=True)

In [7]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="mask-classification",

    # track hyperparameters and run metadata
    config={
    "learning_rate": args.lr,
    "architecture": args.model,
    "epochs": args.epochs,
    "batch_size" : args.batch_size,
    "loss" : args.criterion,
    },
    name=f"{args.model}_{args.criterion}_resize{args.resize}_Project[{project_idx}]_lr[{args.lr}]"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kgw5430. Use `wandb login --relogin` to force relogin


## Data Load & Train

In [7]:
from data.dataset0 import CustomTrainDataset

#-- Data Load & Labeling
train_df = pd.read_csv(TRAIN_CSV_PATH)
train_df['age_label'] = None
train_df['age_label'] = train_df['path'].apply(lambda x : int((str(x).split('_')[3])))
train_df['age_label'].loc[train_df['age_label'] < 30] = 0
train_df['age_label'].loc[train_df['age_label'] >= 30] = 1

In [8]:
model_name = "Age"
age_model, age_best_score, age_best_loss = main(args, train_df, data_dir, model_name, 2)
torch.save(age_model.module.state_dict(), f'{model_dir}_all/Project{project_idx}/{model_name}0_[{args.model}]_[score{age_best_score:.4f}]_[loss{age_best_loss:.4f}].pt')

========================= Age Model Train Start =========================
========================= TRAIN epoch:1 ========================= lr:0.000001000



========================= VALID =========================



Epoch [1], Train Loss : [0.57391] Val Loss : [0.44238] Val ACC : [0.88968]
 
========================= TRAIN epoch:2 ========================= lr:0.000001000



========================= VALID =========================



Epoch [2], Train Loss : [0.34162] Val Loss : [0.22441] Val ACC : [0.93066]
 
========================= TRAIN epoch:3 ========================= lr:0.000001000


...


Epoch [43], Train Loss : [0.05203] Val Loss : [0.05797] Val ACC : [0.98062]
 
Epoch    43: reducing learning rate of group 0 to 6.2500e-08.
========================= TRAIN epoch:44 ========================= lr:0.000000062



========================= VALID =========================



Epoch [44], Train Loss : [0.05150] Val Loss : [0.05536] Val ACC : [0.97694]
 
========================= TRAIN epoch:45 ========================= lr:0.000000062



========================= VALID =========================



Epoch [45], Train Loss : [0.05275] Val Loss : [0.05329] Val ACC : [0.98036]
 
Best Loss : [0.05219] Best ACC : [0.98116]
========================= Age Model Train End =========================


In [10]:
from data.dataset1 import CustomTrainDataset

#-- Data Load & Labeling
train_df = pd.read_csv(TRAIN_CSV_PATH)
train_df['gender_label'], train_df['age_label'] = None, None

_gender_labels = {"male": 0, "female": 1}
train_df['gender_label'] = train_df['path'].apply(lambda x : _gender_labels[str(x).split('_')[1]])

train_df['age_label'] = train_df['path'].apply(lambda x : int((str(x).split('_')[3])))
train_df['age_label'].loc[(train_df['age_label'] >= 30) & (train_df['age_label'] < 60)] = 0
train_df['age_label'].loc[(train_df['age_label'] < 30) & (train_df['age_label'] >= 60)] = 2

In [11]:
model_name = "Age"
age_model, age_best_score, age_best_loss = main(args, train_df, data_dir, model_name, 2)
torch.save(age_model.module.state_dict(), f'{model_dir}_all/Project{project_idx}/{model_name}1_[{args.model}]_[score{age_best_score:.4f}]_[loss{age_best_loss:.4f}].pt')

========================= Age Model Train Start =========================
========================= TRAIN epoch:1 ========================= lr:0.000001000



========================= VALID =========================



Epoch [1], Train Loss : [0.63177] Val Loss : [0.54368] Val ACC : [0.81922]
 
========================= TRAIN epoch:2 ========================= lr:0.000001000



========================= VALID =========================



Epoch [2], Train Loss : [0.48617] Val Loss : [0.40245] Val ACC : [0.85382]
 
========================= TRAIN epoch:3 ========================= lr:0.000001000


...


Epoch [37], Train Loss : [0.15682] Val Loss : [0.20339] Val ACC : [0.91322]
 
Epoch    37: reducing learning rate of group 0 to 1.2500e-07.
========================= TRAIN epoch:38 ========================= lr:0.000000125



========================= VALID =========================



Epoch [38], Train Loss : [0.15545] Val Loss : [0.20684] Val ACC : [0.91178]
 
========================= TRAIN epoch:39 ========================= lr:0.000000125



========================= VALID =========================



Epoch [39], Train Loss : [0.15407] Val Loss : [0.20618] Val ACC : [0.90975]
 
Best Loss : [0.20332] Best ACC : [0.91271]
========================= Age Model Train End =========================


In [8]:
from data.dataset2 import CustomTrainDataset
#-- Data Load & Labeling
train_df = pd.read_csv(TRAIN_CSV_PATH)
train_df['gender_label'], train_df['age_label'] = None, None

_gender_labels = {"male": 0, "female": 1}
train_df['gender_label'] = train_df['path'].apply(lambda x : _gender_labels[str(x).split('_')[1]])

train_df['age_label'] = train_df['path'].apply(lambda x : int((str(x).split('_')[3])))
train_df['age_label'].loc[train_df['age_label'] >= 60] = 0
train_df['age_label'].loc[train_df['age_label'] < 60] = 1

In [9]:
model_name = "Age"
age_model, age_best_score, age_best_loss = main(args, train_df, data_dir, model_name, 2)
torch.save(age_model.module.state_dict(), f'{model_dir}_all/Project{project_idx}/{model_name}2_[{args.model}]_[score{age_best_score:.4f}]_[loss{age_best_loss:.4f}].pt')

========================= Age Model Train Start =========================
========================= TRAIN epoch:1 ========================= lr:0.000001000



========================= VALID =========================



Epoch [1], Train Loss : [0.65408] Val Loss : [0.60523] Val ACC : [0.54885]
 
========================= TRAIN epoch:2 ========================= lr:0.000001000



========================= VALID =========================



Epoch [2], Train Loss : [0.55339] Val Loss : [0.53496] Val ACC : [0.56392]
 
========================= TRAIN epoch:3 ========================= lr:0.000001000


...


Epoch [33], Train Loss : [0.16951] Val Loss : [0.26328] Val ACC : [0.70798]
 
Epoch    33: reducing learning rate of group 0 to 1.2500e-07.
========================= TRAIN epoch:34 ========================= lr:0.000000125



========================= VALID =========================



Epoch [34], Train Loss : [0.16889] Val Loss : [0.26206] Val ACC : [0.71387]
 
========================= TRAIN epoch:35 ========================= lr:0.000000125



========================= VALID =========================



Epoch [35], Train Loss : [0.17116] Val Loss : [0.26343] Val ACC : [0.71140]
 
Best Loss : [0.25870] Best ACC : [0.71210]
========================= Age Model Train End =========================
